In [34]:
#استيراد المكتبات اللازمة
import pandas as pd

In [4]:
# تحميل ملف البيانات في إطار بيانات
df= pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')

In [ ]:
#إظهار الصفوف الأولى من إطار البيانات
print(df.head())

In [ ]:
#إظهار معلومات اطار البيانات
print(df.info())

In [7]:
#حذف الأعمدة غير الضرورية
df.drop(['ORDERNUMBER', 'ORDERLINENUMBER', 'ORDERDATE', 'QTR_ID',
         'MSRP', 'CUSTOMERNAME','PHONE', 'ADDRESSLINE1',
         'ADDRESSLINE2', 'STATE', 'POSTALCODE', 'COUNTRY', 'TERRITORY',
         'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'DEALSIZE'], axis=1, inplace=True)

In [ ]:
#التأكد من قيم اطار البيانات

print(df.info())

In [ ]:
#انشاء مصفوفة للأعمدة الرقمية
#ايجاد المتوسط و معرفةاعلى قيمة واصغر قيمة لكل عمود
#معرفة القيم المتطرفة
numeric_column= df[['QUANTITYORDERED', 'PRICEEACH', 'SALES', 'MONTH_ID', 'YEAR_ID']]

for col in numeric_column:
   desc= df[col].describe()
   print(desc)

In [ ]:
#حساب المقاييس لكل منتج فقط للمنتجات المباعة
#1-إجمالي الإيرادات الناتجة
#2- إجمالي الوحدات المباعة
#3- متوسط سعر الوحدة
#4- إجمالي عدد الطلبات
# Filter out cancelled or on-hold or any unvalid status orders
valid_statuses = ['In Process', 'Shipped']
df_filtered = df[df['STATUS'].isin(valid_statuses)]

# Group by 'PRODUCTCODE' and calculate the required metrics
product_metrics = df_filtered.groupby('PRODUCTCODE').agg(
    total_revenue=('SALES', 'sum'),
    total_units_sold=('QUANTITYORDERED', 'sum'),
    average_price_per_unit=('PRICEEACH', 'mean'),
    total_orders=('PRODUCTCODE', 'count')
).reset_index()

# Display the result
product_metrics.to_html("product_metrics_table.html")

print(product_metrics)

In [ ]:
# Rank products based on each metric
product_metrics['rank_revenue'] = product_metrics['total_revenue'].rank(ascending=False)
product_metrics['rank_units_sold'] = product_metrics['total_units_sold'].rank(ascending=False)
product_metrics['rank_price'] = product_metrics['average_price_per_unit'].rank(ascending=False)
product_metrics['rank_orders'] = product_metrics['total_orders'].rank(ascending=False)

# Create a composite rank by summing up all individual ranks
product_metrics['composite_rank'] = product_metrics[['rank_revenue', 'rank_units_sold', 'rank_price', 'rank_orders']].sum(axis=1)

# Sort by composite rank (lower is better)
product_metrics_sorted = product_metrics.sort_values(by='composite_rank')




# Display the ranked products
print(product_metrics_sorted[['PRODUCTCODE', 'total_revenue', 'total_units_sold', 'average_price_per_unit', 'total_orders', 'composite_rank']])


In [ ]:
#Visualization: Bar plot of top revenue products
import matplotlib.pyplot as plt
import seaborn as sns

# Top 10 products by revenue
top_revenue_products = product_metrics_sorted.head(10)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='total_revenue', y='PRODUCTCODE', data=top_revenue_products, palette='Blues_d')
plt.title('Top 10 Products by Revenue')
plt.xlabel('Total Revenue')
plt.ylabel('Product Code')
plt.savefig("top_revenue_products.png")

plt.show()



In [ ]:
#Visualization: Scatter plot of units sold vs. average price per unit
plt.figure(figsize=(10, 6))
sns.scatterplot(x='average_price_per_unit', y='total_units_sold', size='total_revenue', hue='PRODUCTCODE', data=product_metrics_sorted, sizes=(50, 500))
plt.title('Products: Units Sold vs. Average Price')
plt.xlabel('Average Price per Unit')
plt.ylabel('Total Units Sold')
plt.savefig("Units Sold vs. Average Price.png")

plt.show()


In [ ]:
#Visualization: Top products by average price
# Top 10 products by average price
top_price_products = product_metrics_sorted.sort_values(by='average_price_per_unit', ascending=False).head(10)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='average_price_per_unit', y='PRODUCTCODE', data=top_price_products, palette='Greens_d')
plt.title('Top 10 Products by Average Price per Unit')
plt.xlabel('Average Price per Unit')
plt.ylabel('Product Code')
plt.savefig("Top products by average price.png")

plt.show()


In [ ]:
#Visualization: Bottom 10 products by composite rank
# Bottom 10 products by composite rank
bottom_products = product_metrics_sorted.tail(10)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='composite_rank', y='PRODUCTCODE', data=bottom_products, palette='Reds_d')
plt.title('Bottom 10 Products by Composite Rank')
plt.xlabel('Composite Rank (higher = worse performance)')
plt.ylabel('Product Code')
plt.savefig("Bottom 10 products by composite rank.png")

plt.show()


In [ ]:
#Visualization: Top 10 products by composite rank
# Top 10 products by composite rank
top_composite_products = product_metrics_sorted.head(10)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='composite_rank', y='PRODUCTCODE', data=top_composite_products, palette='Purples_d')
plt.title('Top 10 Products by Composite Rank')
plt.xlabel('Composite Rank (lower = better performance)')
plt.ylabel('Product Code')
plt.savefig("Top 10 products by composite rank.png")

plt.show()


In [ ]:
#حساب المقاييس لكل شهر
#1- إجمالي الإيرادات الناتجة
#2- اجمالي الوحدات المباعة
#3- متوسط سعر الوحدة
# Group by 'YEAR_ID' and 'MONTH_ID' to get the metrics for each month after fillter by status
monthly_metrics = df_filtered.groupby(['YEAR_ID', 'MONTH_ID']).agg(
    total_revenue=('SALES', 'sum'),
    total_units_sold=('QUANTITYORDERED', 'sum')
).reset_index()

# Calculate average price per unit for each month
monthly_metrics['average_price_per_unit'] = monthly_metrics['total_revenue'] / monthly_metrics['total_units_sold']

monthly_metrics.to_html("monthly_metrics_table.html")

# Display the monthly metrics
print(monthly_metrics)


In [ ]:
#Revenue Over Time:
plt.figure(figsize=(10, 6))
sns.lineplot(x='MONTH_ID', y='total_revenue', hue='YEAR_ID', data=monthly_metrics, marker='o')
plt.title('Total Revenue per Month')
plt.xlabel('Month')
plt.ylabel('Total Revenue')
plt.savefig("Revenue Over Time.png")

plt.show()


In [ ]:
#units Sold Over Time:
plt.figure(figsize=(10, 6))
sns.lineplot(x='MONTH_ID', y='total_units_sold', hue='YEAR_ID', data=monthly_metrics, marker='o')
plt.title('Total Units Sold per Month')
plt.xlabel('Month')
plt.ylabel('Total Units Sold')
plt.savefig("units Sold Over Time.png")

plt.show()


In [ ]:
#Average Price Over Time:
plt.figure(figsize=(10, 6))
sns.lineplot(x='MONTH_ID', y='average_price_per_unit', hue='YEAR_ID', data=monthly_metrics, marker='o')
plt.title('Average Price per Unit per Month')
plt.xlabel('Month')
plt.ylabel('Average Price per Unit')
plt.savefig("Average Price Over Time.png")

plt.show()


In [ ]:
#identify the top 5 cities with the highest sales
# Calculate total sales by city
city_sales = df.groupby('CITY').agg(total_sales=('SALES', 'sum')).reset_index()

# Sort by total sales to identify top 5 cities
top_5_cities = city_sales.sort_values(by='total_sales', ascending=False).head(5)
other_cities = city_sales[~city_sales['CITY'].isin(top_5_cities['CITY'])]

# Display the top 5 cities with the highest sales
print(top_5_cities)


In [ ]:
#Visualization Top 5 Cities by Sales:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the top 5 cities
plt.figure(figsize=(10, 6))
sns.barplot(x='total_sales', y='CITY', data=top_5_cities, palette='Blues_d')
plt.title('Top 5 Cities by Sales')
plt.xlabel('Total Sales')
plt.ylabel('City')
plt.savefig("Top 5 Cities by Sales.png")

plt.show()
